In [ ]:
import pyodbc
drivers=[item for item in pyodbc.drivers()]
print("drivers:{}".format(drivers))

In [ ]:
server='localhost\\SQLEXP'
database='Serbian_elections'
username='KK'
password='kk00'
driver='{ODBC Driver 17 for SQL Server}'

In [ ]:
connection=pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password+';Encrypt=no')

In [ ]:
cursor=connection.cursor()

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
URL = "https://en.wikipedia.org/wiki/2023_Serbian_parliamentary_election"
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, "html.parser")
citationText = soup.find_all("span", attrs={'class':'reference-text'})

for i in citationText:
    print(i.get_text(strip=True))       

texts = [span.get_text(strip=True) for span in soup.find_all("span", attrs={'class':'reference-text'})]

df = pd.DataFrame(texts)

df.to_csv("wiki2.csv",encoding="utf-8-sig",index=False)
df.to_json("wiki2.json",force_ascii=False)

In [ ]:
if webpage.status_code == 200:

    tables = soup.find_all('table')

    for table in tables:
        if 'sortable' in table.attrs.get('class', []):
            df_se = pd.read_html(str(table))[0]

            print(df_se)
            break
    else:
        print("No suitable table with the specified class found.")

else:
    print("Failed to retrieve the webpage. Check the URL or your internet connection.")


In [ ]:
df_se = df_se.drop(df_se.columns[0], axis=1).iloc[:-6]

print(df_se)

In [ ]:
df_se.to_csv('election_results_se.csv', index=False)
df_se.to_json("election_results_se.json",force_ascii=False)

print("Table successfully scraped and saved as 'election_results_se.csv'")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Flatten the MultiIndex column names
df_se.columns = [' '.join(col).strip() for col in df_se.columns.values]

# Convert numeric columns to appropriate data types
df_se['Votes'] = pd.to_numeric(df_se['99.63% reporting Votes'], errors='coerce')
df_se['Percentage'] = pd.to_numeric(df_se['99.63% reporting %'], errors='coerce')
df_se['Change'] = pd.to_numeric(df_se['99.63% reporting +/–'], errors='coerce')
df_se['Seats'] = pd.to_numeric(df_se['99.63% reporting Seats'], errors='coerce')
df_se['Change_Seats'] = pd.to_numeric(df_se['99.63% reporting +/–.1'], errors='coerce')

# Sorting DataFrame by Votes
df_se = df_se.sort_values(by='Votes', ascending=False)

# Plotting
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))

# Bar plot for Votes
df_se.plot(kind='bar', x='99.63% reporting Party, alliance, or citizens\' group.1', y='Votes', ax=ax1, color='blue', legend=False)
ax1.set_ylabel('Votes')
ax1.set_title('Votes by Party (Sorted)')

# Bar plot for Seats
df_se.plot(kind='bar', x='99.63% reporting Party, alliance, or citizens\' group.1', y='Seats', ax=ax2, color='green', legend=False)
ax2.set_ylabel('Seats')
ax2.set_title('Seats by Party (Sorted)')

# Line plot for Percentage
df_se.plot(kind='line', x='99.63% reporting Party, alliance, or citizens\' group.1', y='Percentage', marker='o', ax=ax3, color='red', legend=False)
ax3.set_ylabel('Percentage')
ax3.set_title('Percentage by Party (Sorted)')

# Adjust layout
plt.tight_layout()

# Save the plots to files
plt.savefig('votes_by_party.png')
plt.savefig('seats_by_party.png')
plt.savefig('percentage_by_party.png')

# Show the plots
plt.show()

print("Graphs exported.")


In [ ]:
URL_PL = "https://en.wikipedia.org/wiki/2023_Polish_parliamentary_election"
webpage_PL = requests.get(URL_PL)
soup2 = BeautifulSoup(webpage_PL.text, "html.parser")
citationText = soup.find_all("span", attrs={'class':'reference-text'})

for i in citationText:
    print(i.get_text(strip=True))       

texts = [span.get_text(strip=True) for span in soup.find_all("span", attrs={'class':'reference-text'})]

df2 = pd.DataFrame(texts)

df2.to_csv("text_pl.csv",encoding="utf-8-sig",index=False)
df2.to_json("text_pl.json",force_ascii=False)

In [ ]:
if webpage_PL.status_code == 200:
    soup = BeautifulSoup(webpage_PL.content, 'lxml')

    tables = soup.find_all('table')

    if 0 <= 15 < len(tables):
        desired_table = tables[15]

        df_pl = pd.read_html(str(desired_table))[0]

        print(df_pl)
    else:
        print("Error: Desired table index is out of range.")
else:
    print(f"Error: Unable to fetch the page (Status Code: {webpage_PL.status_code})")

In [ ]:
#porządkowanie danych
df_pl = df_pl.drop(df_pl.columns[[0, 2]], axis=1).iloc[:-6]

print(df_pl)


In [ ]:
df_pl.to_csv('election_results_pl.csv', index=False)
df_pl.to_json("election_results_pl.json",force_ascii=False)

print("Table successfully scraped and saved as 'election_results_pl.csv'")

In [ ]:
# zmiana formatu wartości
df_pl['Votes'] = pd.to_numeric(df_pl['Votes'], errors='coerce')
df_pl['Seats'] = pd.to_numeric(df_pl['Seats'], errors='coerce')

# grupowanie wartości dla partii
grouped_df = df_pl.groupby('Party or alliance.1')[['Votes', 'Seats']].sum()

# sortowanie
grouped_df = grouped_df.sort_values(by='Votes', ascending=False)

# plotting
fig, ax1 = plt.subplots(figsize=(12, 6))

# wykres dla głosów
grouped_df.plot(kind='bar', y='Votes', ax=ax1, color='blue', legend=False)
ax1.set_ylabel('Votes')

# oś dla mandatów
ax2 = ax1.twinx()
grouped_df.plot(kind='line', y='Seats', ax=ax2, color='red', marker='o', legend=False)
ax2.set_ylabel('Seats')

plt.title('Votes and Seats by Party or Alliance')
plt.xlabel('Party or Alliance')

plt.savefig('votes_and_seat_by_party_pl.png')

plt.show()

print("Graph exported.")